<h1> Data Quality - Análise e Monitoramento </h1>
<br>Anderson Oliveira
<br>set/2020

<h3>Passos</h3>
<br>
1- Instalando a libs

2- Análise dos dados

    Entendendo a base (data profiling / Documentação)
    Definindo as dimensões a serem aplicadas (repositório de regras)
3- Monitoramento dos dados

    Criando os componentes para executar as avaliações (great_expectations)
    Gerando um dashboard para acompanhamento (map)

<h3>1- Instalando as libs</h3>

In [ ]:
pip install pandas

In [ ]:
pip install great_expectations # https://greatexpectations.io/

<h3>2- Análise de dados</h3> 

<h6>Antes de definir as regras que serão aplicadas no processo de data quality, é importante conhecer o perfil da base/tabela que iremos aferir qualidade. Para isso podemos analisar o profiling (conforme citado na aula de profiling) juntamente com a documentação (metadados) da base. Após esta análise iremos compor nosso repositório de regras <br><br>
Nota: nesta aula iremos abordar apenas as dimensões Completude, Unicidade, Consistência e Razoabilidade.</h6>

In [ ]:
import pandas as pd

data = {'nome_coluna':  ['cd_ibge', 'id_familia', 'cod_calcamento_domic_fam', 'vlr_renda_media_fam'],# coluna que será avaliada
        'dimensao': ['Completude', 'Unicidade', 'Consistencia', 'Razoabilidade'],# dimensão avaliada na coluna
        'parametro':[None, None, [1,2,3], {'min': 0, 'max': 500}], # parâmetro definido para algumas dimensões
        'percentual_esperado_corretos':[100, 95, 100, 85]} # percentual esperado de conformidades

df_rules = pd.DataFrame (data, columns = ['nome_coluna','dimensao', 'parametro','percentual_esperado_corretos'])
display(df_rules)

In [ ]:
import pandas as pd

data = {'nome_coluna':  ['cd_ibge', 'id_familia', 'cod_calcamento_domic_fam', 'vlr_renda_media_fam', 'cd_ibge', 'cod_material_piso_fam', 'cod_material_piso_fam', 'cod_material_piso_fam', 'cod_agua_canalizada_fam','peso.fam', 'nom_estab_assist_saude_fam'],
        'dimensao': ['Completude', 'Unicidade', 'Consistencia', 'Razoabilidade', 'Unicidade', 'Completude', 'Unicidade', 'Consistencia', 'Completude', 'Razoabilidade', 'Completude'],
        'parametro':[None, None, [1,2,3], {'min': 0, 'max': 500}, None, None, None, [True, False], None, {'min': 4, 'max': 10}, None],
        'percentual_esperado_corretos':[100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100]}

df_rules = pd.DataFrame (data, columns = ['nome_coluna','dimensao', 'parametro','percentual_esperado_corretos'])

<h3>3- Monitoramento dos dados</h3>

<h4>Criando os componentes para executar as avaliações (great_expectations)</h4>

In [ ]:
import great_expectations as ge
import json

df_db = pd.read_csv('base_bolsa_familia.csv',';')

df_db = ge.dataset.PandasDataset(df_db)

In [ ]:
df_rules['json_result'] = df_rules.apply(lambda row: 
                                  df_db.expect_column_values_to_not_be_null(row.nome_coluna).to_json_dict() 
                                  if row.dimensao == 'Completude' 
                                  else 
                                    (df_db.expect_column_values_to_be_unique(row.nome_coluna).to_json_dict() 
                                     if row.dimensao == 'Unicidade' 
                                     else 
                                      (df_db.expect_column_values_to_be_in_set(row.nome_coluna,row.parametro).to_json_dict()
                                       if row.dimensao == 'Consistencia'
                                       else 
                                        (df_db.expect_column_values_to_be_between(row.nome_coluna, row.parametro['min'], row.parametro['max']).to_json_dict()
                                         if row.dimensao == 'Razoabilidade'
                                         else None))), axis = 1)

df_rules['resultado']= df_rules.apply(lambda row: 
                                       True if 100 - row.json_result['result']['unexpected_percent'] >= row.percentual_esperado_corretos
                                            else False, axis=1)
display(df_rules)

<h4>Gerando um dashboard para acompanhamento (map)</h4>

In [ ]:
#df = df_rules.groupby('dimensao')['dimensao'].count().reset_index(name='Count')
por_tabela = df_rules.groupby(["resultado"])['resultado'].count().reset_index(name='Count')
por_tabela = pd.pivot_table(por_tabela, values = 'Count', index=['resultado'], columns = 'resultado')
por_dimensao = df_rules.groupby(["dimensao", "resultado"])['resultado'].count().reset_index(name='Count')
por_dimensao = pd.pivot_table(por_dimensao, values = 'Count', index=['dimensao'], columns = 'resultado')
por_coluna = df_rules.groupby(["nome_coluna", "resultado"])['resultado'].count().reset_index(name='Count')
por_coluna = pd.pivot_table(por_coluna, values = 'Count', index=['nome_coluna'], columns = 'resultado')

In [ ]:
ax = por_tabela.plot.barh(rot=0)

In [ ]:
ax = por_dimensao.plot.barh(rot=0)

In [ ]:
ax = por_coluna.plot.barh(rot=0)